In [22]:
import pandas as pd
import altair as alt
import numpy as np

pd.set_option('display.max_columns', None)
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
df = pd.read_csv("../data/raw/SpeedDating.csv",delimiter=",")

In [5]:
df.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,fun7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,1,11.0,0,0.14,0,27.0,2.0,35.0,20.0,20.0,20.0,0.0,5.0,0,6.0,8.0,8.0,8.0,8.0,6.0,7.0,4.0,2.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,2,12.0,0,0.54,0,22.0,2.0,60.0,0.0,0.0,40.0,0.0,0.0,0,7.0,8.0,10.0,7.0,7.0,5.0,8.0,4.0,2.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,7.0,8.0,7.0,8.0,5.0,6.0,7.0,5.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,3,13.0,1,0.16,1,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,1,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,5.0,8.0,9.0,8.0,5.0,7.0,7.0,NaN,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,4,14.0,1,0.61,0,23.0,2.0,30.0,5.0,15.0,40.0,5.0,5.0,1,7.0,8.0,9.0,8.0,9

In [16]:
df_time0 = df[["gender", "condtn", "wave", "round", "match", "samerace", "age_o", "race_o", "dec_o", "attr_o"]]
df_time0.rename(columns = 
{
    "round": "numberInWave",
    "age_o": "ageOfPartner",
    "race_o": "raceOfPartner",
    "dec_o": "decisionOfPartner",
    "attr_o": "ratingOfPartner"
}, inplace = True)

df_time0["gender"] = df_time0["gender"].astype('category')
df_time0["condtn"] = df_time0["condtn"].astype('category')
df_time0["match"] = df_time0["match"].astype('category')
df_time0["samerace"] = df_time0["samerace"].astype('category')
df_time0["raceOfPartner"] = df_time0["raceOfPartner"].astype('category')
df_time0["decisionOfPartner"] = df_time0["decisionOfPartner"].astype('category')

df_time0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   gender             8378 non-null   category
 1   condtn             8378 non-null   category
 2   wave               8378 non-null   int64   
 3   numberInWave       8378 non-null   int64   
 4   match              8378 non-null   category
 5   samerace           8378 non-null   category
 6   ageOfPartner       8274 non-null   float64 
 7   raceOfPartner      8305 non-null   category
 8   decisionOfPartner  8378 non-null   category
 9   ratingOfPartner    8166 non-null   float64 
dtypes: category(6), float64(2), int64(2)
memory usage: 311.8 KB


C:\Users\lukas\AppData\Local\Temp\ipykernel_17028\1427666880.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time0.rename(columns =
C:\Users\lukas\AppData\Local\Temp\ipykernel_17028\1427666880.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time0["gender"] = df_time0["gender"].astype('category')
C:\Users\lukas\AppData\Local\Temp\ipykernel_17028\1427666880.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [17]:
df_time0

,gender,condtn,wave,numberInWave,match,samerace,ageOfPartner,raceOfPartner,decisionOfPartner,ratingOfPartner
0,0,1,1,10,0,0,27.0,2.0,0,6.0
1,0,1,1,10,0,0,22.0,2.0,0,7.0
2,0,1,1,10,1,1,22.0,4.0,1,10.0
3,0,1,1,10,1,0,23.0,2.0,1,7.0
4,0,1,1,10,1,0,24.0,3.0,1,8.0
...,...,...,...,...,...,...,...,...,...,...
8373,1,2,21,22,0,0,26.0,3.0,1,10.0
8374,1,2,21,22,0,0,24.0,6.0,0,6.0
8375,1,2,21,22,0,0,29.0,3.0,0,2.0
8376,1,2,21,22,0,0,22.0,4.0,1,5.0


In [34]:
def renderBar(chart, colName):
    return chart.encode(alt.X(colName),
        alt.Y("count(" + colName + ")"),
        alt.Tooltip("count(" + colName + ")"))

In [40]:
chart = alt.Chart(df_time0).mark_bar().interactive().properties(
    width=300,
    height=200
)

renderBar(chart, "gender") & renderBar(chart, "condtn") & renderBar(chart, "wave") & renderBar(chart, "numberInWave") | \
renderBar(chart, "ageOfPartner") & renderBar(chart, "raceOfPartner") & renderBar(chart, "samerace") | \
renderBar(chart, "match") & renderBar(chart, "decisionOfPartner") & renderBar(chart, "ratingOfPartner")

alt.HConcatChart(...)

Interesting fields:

- gender (Female = 0, Male = 1)
- condtn (Limited choice = 1, extensive choice = 2)
- wave (1-21), especially because of variations
- round (number of people in wave, if not always 10)
- order (number of date that night)
- match (yes = 1, no = 0)
- samerace (yes = 1, no = 0)
- age_o (age of partner)
- race_o (race of partner)
- dec_o (decision of partner the night of event)
- attr_o (rating by partner the night of the event, for all 6 attributes)

Field at time 1 (signup time):
- age
- field (of study)
- undergrd (bachelor student)
- mn_sat (proxy for intelligence)
- race (Black/African American=1, European/Caucasian-American=2, Latino/Hispanic American=3, Asian/Pacific Islander/Asian-American=4, Native American=5, Other=6)
- imprace (importance of same racial/ethic background)
- imprelig (importance of same religion)
- from + zipcode (originally from)
- income (Median household income based on zip code)
- goal (Seemed like a fun night out=1, To meet new people=2, To get a date=3, Looking for a serious relationship=4, To say I did it=5, Other=6)
- date (Several times a week=1, Twice a week=2, Once a week=3, Twice a month=4, Once a month=5, Several times a year=6, Almost never=7)
- go out (Several times a week=1, Twice a week=2, Once a week=3, Twice a month=4, Once a month=5, Several times a year=6, Almost never=7)
- career
- interest in activities from 1-10
- exphappy from 1-10 (how happy do you expect to be with prople you meet during the speed-dating event)
- expnum (out of 20 people, how many do you expect to be interested in you?)

Fields at time 2 (follow up):
- satis_2 (how satisfied, 1 - 10)

Field at time 3 (followup2, 3-4 after they knew their matches):
- you_call (how many have you contacted)
- them_cal (how many contacted you)
- date_3 (have you been on a date? yes = 1, no = 2)
- numdat_3 (how many dates?)
- num_in_3
